In [27]:
import pandas as pd
import yfinance as yf
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import plotly.graph_objects as go

Los datos de los 5 factores Fama-French se tomarán para los años 2020 a 2024, es decir del 2020-01-01 al 2024-12-31.

In [28]:
start_date = "2015-01-01"
end_date = "2025-11-04"

In [29]:
#Cargando los 10 ETF Sectoriales de S&P500
#XLC
etfs_name = ['XLF', 'XLK', 'XLV', 'XLE', 'XLI', 'XLY', 'XLP', 'XLU', 'XLRE', 'XLB']

efts_data = []

for name in etfs_name:
    
    data = yf.download(name, start=start_date, end=end_date)
    
    if data.empty:
            print(f"No hay datos para {name}")
    else:
        data.index = data.index.normalize()

        #Modificando para obtener la data mensual
        monthly_data = data[['Close']].resample('M').last()
        monthly_data[name] = monthly_data['Close'].pct_change()
        efts_data.append(monthly_data[name])

C:\Users\crdie\AppData\Local\Temp\ipykernel_15988\530410504.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\crdie\AppData\Local\Temp\ipykernel_15988\530410504.py:17: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\crdie\AppData\Local\Temp\ipykernel_15988\530410504.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\crdie\AppData\Local\Temp\ipykernel_15988\530410504.py:17: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\crdie\AppData\Local\Temp\ipykernel_15988\530410504.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\crdie\AppD

In [30]:
#Uniendo todos los datasets en uno solo
asset_returns_df = pd.concat(efts_data, axis=1)
asset_returns_df.index = pd.to_datetime(asset_returns_df.index)

#Modificando el date a formato yyyymm
asset_returns_df.index = asset_returns_df.index.strftime('%Y%m')
asset_returns_df

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB
Date,,,,,,,,,,
201501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201502,0.058235,0.079950,0.042876,0.045930,0.053509,0.085441,0.041441,-0.063949,NaN,0.079681
201503,-0.006160,-0.034356,0.006406,-0.011552,-0.025461,-0.004830,-0.019427,-0.009953,NaN,-0.049049
201504,0.000829,0.027510,-0.010896,0.065739,-0.002510,-0.000531,-0.007591,-0.004727,NaN,0.033620
201505,0.019478,0.018554,0.045043,-0.051887,0.003236,0.013146,0.008683,0.006332,NaN,0.003769
...,...,...,...,...,...,...,...,...,...,...
202507,0.000000,0.037555,-0.032347,0.028299,0.030436,0.018865,-0.014697,0.049106,-0.000241,-0.000911
202508,0.030934,-0.001104,0.053669,0.036464,0.000000,0.046561,0.012534,-0.015758,0.021734,0.051864
202509,0.001062,0.075337,0.017292,-0.003182,0.018796,0.035914,-0.023177,0.041236,0.003346,-0.024217


In [31]:
# Cargando los 5 Fama-French factores

raw_data_ff = pd.read_csv('MonthlyFF4F.csv', skiprows=3)

#Pequeña limpieza de nombre de columnas
ff_cols = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
fama_french_df = raw_data_ff.rename(columns={raw_data_ff.columns[0]: 'Date'})
fama_french_df.set_index('Date', inplace=True)
fama_french_df[ff_cols] = fama_french_df[ff_cols] / 100
fama_french_df.index = fama_french_df.index.astype(str)
fama_french_df

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
196307,-0.0039,-0.0048,-0.0081,0.0064,-0.0115,0.0027
196308,0.0508,-0.0080,0.0170,0.0040,-0.0038,0.0025
196309,-0.0157,-0.0043,0.0000,-0.0078,0.0015,0.0027
196310,0.0254,-0.0134,-0.0004,0.0279,-0.0225,0.0029
196311,-0.0086,-0.0085,0.0173,-0.0043,0.0227,0.0027
...,...,...,...,...,...,...
202503,-0.0639,-0.0149,0.0290,0.0211,-0.0047,0.0034
202504,-0.0084,-0.0186,-0.0340,-0.0285,-0.0267,0.0035
202505,0.0606,-0.0072,-0.0288,0.0126,0.0251,0.0038


In [ ]:
#Se hace un inner join para mentener informacion solo de las fechas dadas
final_df = asset_returns_df.join(fama_french_df,on='Date')

final_df = final_df.dropna()

final_df

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,,,,,,,,,,
201511,0.019933,0.007102,-0.003224,-0.000147,0.008660,-0.002347,-0.009222,-0.021257,-0.006462,0.009938,0.0057,0.0330,-0.0043,-0.0265,-0.0106,0.0000
201512,-0.023644,-0.020812,0.017228,-0.105170,-0.025724,-0.028431,0.029177,0.021192,0.025327,-0.043401,-0.0215,-0.0295,-0.0257,0.0034,0.0012,0.0001
201601,-0.088544,-0.037123,-0.077190,-0.034980,-0.056970,-0.051817,0.005348,0.049446,-0.063943,-0.107094,-0.0574,-0.0344,0.0208,0.0278,0.0305,0.0001
201602,-0.029005,-0.006547,-0.003611,-0.028002,0.042608,0.004453,0.003152,0.019375,0.004796,0.078153,-0.0007,0.0087,-0.0061,0.0331,0.0194,0.0002
201603,0.071591,0.088217,0.027075,0.101659,0.069735,0.066974,0.047383,0.079765,0.106971,0.075731,0.0695,0.0100,0.0122,0.0068,0.0001,0.0002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202503,-0.041952,-0.082860,-0.015813,0.034932,-0.035630,-0.083147,-0.011764,0.002394,-0.023920,-0.026703,-0.0639,-0.0149,0.0290,0.0211,-0.0047,0.0034
202504,-0.021080,0.016902,-0.037943,-0.138577,0.001144,-0.001013,0.001959,0.000634,-0.013142,-0.024308,-0.0084,-0.0186,-0.0340,-0.0285,-0.0267,0.0035
202505,0.045119,0.099728,-0.055741,0.012795,0.088401,0.083849,0.012220,0.038276,0.010412,0.029205,0.0606,-0.0072,-0.0288,0.0126,0.0251,0.0038


In [33]:
#Calculando los excedentes
excess_returns_df = final_df[etfs_name].subtract(final_df['RF'], axis=0)

excess_returns_df

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB
Date,,,,,,,,,,
201511,0.019933,0.007102,-0.003224,-0.000147,0.008660,-0.002347,-0.009222,-0.021257,-0.006462,0.009938
201512,-0.023744,-0.020912,0.017128,-0.105270,-0.025824,-0.028531,0.029077,0.021092,0.025227,-0.043501
201601,-0.088644,-0.037223,-0.077290,-0.035080,-0.057070,-0.051917,0.005248,0.049346,-0.064043,-0.107194
201602,-0.029205,-0.006747,-0.003811,-0.028202,0.042408,0.004253,0.002952,0.019175,0.004596,0.077953
201603,0.071391,0.088017,0.026875,0.101459,0.069535,0.066774,0.047183,0.079565,0.106771,0.075531
...,...,...,...,...,...,...,...,...,...,...
202503,-0.045352,-0.086260,-0.019213,0.031532,-0.039030,-0.086547,-0.015164,-0.001006,-0.027320,-0.030103
202504,-0.024580,0.013402,-0.041443,-0.142077,-0.002356,-0.004513,-0.001541,-0.002866,-0.016642,-0.027808
202505,0.041319,0.095928,-0.059541,0.008995,0.084601,0.080049,0.008420,0.034476,0.006612,0.025405


In [34]:
fig = px.scatter_matrix(excess_returns_df, width=1200, height=900, title='<b>Scatter matrix para los rendimientos en exceso de los EFTs</b>')

fig.update_layout(title_x = 0.5)

fig.show()

In [35]:
#Matriz de correlacion

correlation_matrix = excess_returns_df.corr()

# heatmap con Plotly
fig = px.imshow(
    correlation_matrix,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1,
    aspect="auto",
    text_auto=".2f"
)

fig.update_layout(
    title='<b>Matriz de Correlación para los rendimientos en exceso de los EFTs </b>',
    width=800,
    height=800,
    xaxis_title="Variables",
    yaxis_title="Variables",
    title_x =0.5
)

fig.show()

Hay diversas variables que tienen una correlación bastante alta $(>0.8)$, se puede ver como los siguientes grupos de variables mantienen una correlación alta entre sí:

- Grupo 1: XLB, XLI, XLF

- Grupo 2: XLK, XLY

In [ ]:
#Estandarizacion
scaler = StandardScaler()

excess_scaled = scaler.fit_transform(excess_returns_df)
excess_scaled = pd.DataFrame(excess_scaled, columns=excess_returns_df.columns, index=excess_returns_df.index)

excess_scaled

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB
Date,,,,,,,,,,
201511,0.173940,-0.166713,-0.218118,-0.090160,-0.028223,-0.204278,-0.400486,-0.661955,-0.229448,0.050256
201512,-0.594578,-0.671773,0.276072,-1.306561,-0.667421,-0.651216,0.658533,0.302949,0.405649,-0.932490
201601,-1.736532,-0.965851,-2.016573,-0.494379,-1.246587,-1.050400,-0.000371,0.946707,-1.383446,-2.103793
201602,-0.690679,-0.416403,-0.232367,-0.414794,0.597318,-0.091624,-0.063857,0.259272,-0.007819,1.301049
201603,1.079364,1.292124,0.512730,1.085553,1.100133,0.975566,1.159205,1.635239,2.039897,1.256497
...,...,...,...,...,...,...,...,...,...,...
202503,-0.974796,-1.849958,-0.606374,0.276409,-0.912187,-1.641510,-0.564798,-0.200546,-0.647461,-0.686098
202504,-0.609297,-0.053117,-1.146135,-1.732466,-0.232415,-0.241250,-0.188087,-0.242919,-0.433467,-0.643898
202505,0.550230,1.434764,-1.585607,0.015625,1.379388,1.202152,0.087355,0.607915,0.032572,0.334688


In [ ]:
#Aplicando PCA
pca = PCA()

pca.fit_transform(excess_scaled)

array([[-0.50024191, -0.62703672,  0.0071926 , ..., -0.00771596,
        -0.1681907 , -0.03774108],
       [-1.03876488,  1.712704  ,  0.00923292, ..., -0.34749976,
         0.21390752,  0.07230229],
       [-3.42332948,  1.4634436 , -0.35598929, ..., -0.03566909,
         0.56798821,  0.2340463 ],
       ...,
       [ 1.33301328, -0.48135291,  1.34649156, ...,  0.10696925,
        -0.04058605,  0.4068602 ],
       [ 0.71161905, -0.92253458,  0.67670832, ..., -0.51401439,
        -0.61100271,  0.12239506],
       [-0.22134695,  0.07553847,  0.28935013, ..., -0.0726354 ,
        -0.07702173,  0.36620703]])

In [38]:
print("Eigenvalores (varianzas explicadas):", pca.explained_variance_)
print("Eigenvectores (componentes principales):\n", pca.components_)
print("Varianza explicada (%):", pca.explained_variance_ratio_ * 100)

Eigenvalores (varianzas explicadas): [6.55795323 1.12363046 0.76858401 0.46641203 0.30549018 0.27397528
 0.2246915  0.16369812 0.12171459 0.0800575 ]
Eigenvectores (componentes principales):
 [[ 0.33400902  0.30754796  0.31280008  0.25681852  0.36481837  0.32715066
   0.30469029  0.23937134  0.33198422  0.35997812]
 [-0.31430813 -0.10404835  0.0783262  -0.42065113 -0.16131711 -0.16452487
   0.38676935  0.63097613  0.3086327  -0.10599428]
 [-0.17843027  0.58088313 -0.11861385 -0.54857988 -0.0181936   0.48580159
  -0.21227141 -0.14748881  0.09102648 -0.06554531]
 [-0.07047419 -0.06174901  0.78735415 -0.29873186 -0.05162303 -0.15004857
   0.17563616 -0.47076617 -0.04870521  0.04508442]
 [-0.30039585  0.44934403 -0.04874141  0.39380975 -0.13067171 -0.01008091
   0.56649082 -0.12425619 -0.25867407 -0.36048575]
 [ 0.36120821 -0.02357296 -0.35164921 -0.45219807  0.39653638 -0.1819902
   0.45587998 -0.12311693 -0.34523946  0.09108532]
 [ 0.12980133 -0.2665311  -0.27812343  0.01760293 -0.146126

In [42]:
explained_var = pca.explained_variance_ratio_
cum_var = np.cumsum(explained_var)
components = np.arange(1, len(explained_var)+1)

# Gráfico Plotly
fig_var = go.Figure()

# Varianza explicada por componente
fig_var.add_trace(go.Bar(
    x=components,
    y=explained_var,
    name='Varianza individual',
    marker_color='royalblue'
))

# Varianza acumulada
fig_var.add_trace(go.Scatter(
    x=components,
    y=cum_var,
    name='Varianza acumulada',
    mode='lines+markers',
    line=dict(color='firebrick', width=3),
    marker=dict(size=8)
))

fig_var.update_layout(
    title='Varianza explicada por componente y varianza acumulada',
    xaxis_title='Componente principal',
    yaxis_title='Fracción de varianza',
    yaxis=dict(range=[0,1.05]),
    #legend=dict(x=0.7, y=0.75),
    width=900,
    height=700
)

fig_var.show()

In [45]:
#Calculando los loadings

eigenvectors = pca.components_
eigenvalues = pca.explained_variance_

loadings = eigenvectors * np.sqrt(eigenvalues[:, np.newaxis])

loadings_df =  pd.DataFrame(loadings, columns=excess_scaled.columns, index= [f"PC{i}" for i in range(1, 11)])

loadings_df

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB
PC1,0.855347,0.787584,0.801034,0.657674,0.934245,0.837784,0.780266,0.612994,0.850162,0.921850
PC2,-0.333171,-0.110293,0.083027,-0.445896,-0.170998,-0.174399,0.409981,0.668844,0.327155,-0.112355
PC3,-0.156428,0.509254,-0.103987,-0.480934,-0.015950,0.425897,-0.186096,-0.129302,0.079802,-0.057463
PC4,-0.048130,-0.042171,0.537719,-0.204017,-0.035256,-0.102475,0.119950,-0.321507,-0.033263,0.030790
PC5,-0.166032,0.248358,-0.026940,0.217663,-0.072224,-0.005572,0.313106,-0.068678,-0.142972,-0.199245
PC6,0.189066,-0.012339,-0.184063,-0.236692,0.207558,-0.095258,0.238620,-0.064443,-0.180707,0.047676
PC7,0.061528,-0.126340,-0.131835,0.008344,-0.069266,0.068632,0.137500,-0.223149,0.317166,-0.093219
PC8,-0.241988,-0.117443,-0.066165,0.022384,0.033016,0.118736,0.074520,-0.058484,-0.047864,0.244981
PC9,0.052466,-0.150989,0.059221,-0.012369,-0.003026,0.236407,0.037300,0.056363,-0.114219,-0.137813
PC10,-0.103942,-0.025752,0.015708,0.014755,0.225556,-0.025152,-0.043310,-0.012613,0.048094,-0.110771


La primer componente contiene los loadings de los EFTs altos y positivos, por lo que se mueven en la misma dirección, esto se interpreta como el mercado en general, pues cuando el "mercado" en general sube, todos suben o bajan. Sin embargo hay algunos un poco mas altos como 

- XLI (0.93): Industriales
- XLU (0.92): Finanzas
- XLE (0.65): Energía

La segunda componente muestra otro grupo, ya que separa a los componentes en dos sectores:

- Positivos con XLU (Utilidades), XLP (Consumo Básico), XLRE (Bienes raices)
- Negativos con XLE (Energía), XLF (Finanzas)

La tercera componente tiene menos componentes representativas, sin embargo con PCA se logró encontrar poca información de la interpretación de las variables, ya que a partir del 3er grupo es mucho ruido y complicado de determinar si realmente es un riesgo de mercado o un patrón estadístico únicamente.